In [7]:
# Abrir o arquivo ETAPA3.txt e processar as linhas
with open('ETAPA3.txt', 'r') as file:
    lines = file.readlines()

# Filtrar as linhas que não começam com [SUM]
filtered_lines = [line for line in lines if not line.startswith('[SUM]')]

# Escrever as linhas filtradas de volta no arquivo
with open('ETAPA3.txt', 'w') as file:
    file.writelines(filtered_lines)

In [8]:
import csv

# Abrir o arquivo de entrada e o arquivo CSV de saída
with open('ETAPA3.txt', 'r') as infile, open('ETAPA3.csv', 'w', newline='') as outfile:
    csv_writer = csv.writer(outfile)
    lines = infile.readlines()

    # Variáveis para controle
    header_found = False
    header = ['ID', 'Interval', 'Transfer', 'Bandwidth']

    for line in lines:
        # Identificar o início de uma nova seção
        if line.startswith("Starting IMUNES simulation with scenario ID:"):
            scenario_id = line.split(":")[-1].strip()  # Extrair o ID do cenário
            execution_value = scenario_id[-1]  # Pegar o último dígito do ID
            if not header_found:
                csv_writer.writerow(['Execution'] + header)  # Adicionar a nova coluna ao cabeçalho
                header_found = True
            continue

        # Processar linhas que contêm dados
        if line.startswith('[') and 'sec' in line and 'Bytes' in line and 'bits/sec' in line:
            parts = line.split()
            id_col = parts[0] + parts[1]  # Combinar '[ ID]'
            interval = parts[2]  # Intervalo de tempo
            transfer = parts[4] + ' ' + parts[5]  # Transferência
            bandwidth = parts[6] + ' ' + parts[7]  # Largura de banda
            csv_writer.writerow([execution_value, id_col, interval, transfer, bandwidth])

In [9]:
import csv

# Abrir o arquivo CSV de entrada e criar um novo arquivo CSV de saída
with open('ETAPA3.csv', 'r') as infile, open('ETAPA3_modified.csv', 'w', newline='') as outfile:
    csv_reader = csv.reader(infile)
    csv_writer = csv.writer(outfile)

    for row in csv_reader:
        # Remover colchetes das colunas
        row = [value.replace('[', '').replace(']', '') for value in row]

        # Modificar a coluna Interval
        if row[2] != 'Interval':  # Ignorar o cabeçalho
            interval_parts = row[2].split('-')
            if len(interval_parts) == 2:
                row[2] = interval_parts[1]  # Manter apenas o segundo valor

        # Converter valores de Transfer e Bandwidth
        if row[3] != 'Transfer' and row[4] != 'Bandwidth':  # Ignorar o cabeçalho
            # Converter Transfer
            if 'KBytes' in row[3]:
                transfer_value = float(row[3].replace('KBytes', '').strip()) / 1024
                row[3] = f"{transfer_value:.3f} MBytes"
            elif 'MBytes' in row[3]:
                row[3] = row[3]  # Já está em MBytes, manter como está

            # Converter Bandwidth
            if 'Kbits/sec' in row[4]:
                bandwidth_value = float(row[4].replace('Kbits/sec', '').strip()) / 1024
                row[4] = f"{bandwidth_value:.3f} Mbits/sec"
            elif 'Mbits/sec' in row[4]:
                row[4] = row[4]  # Já está em Mbits/sec, manter como está

        # Escrever a linha modificada no novo arquivo
        csv_writer.writerow(row)

In [10]:
# Abrir o arquivo CSV modificado para leitura e criar um novo arquivo sem as linhas indesejadas
with open('ETAPA3_modified.csv', 'r') as infile, open('ETAPA3_filtered.csv', 'w', newline='') as outfile:
    csv_reader = csv.reader(infile)
    csv_writer = csv.writer(outfile)

    for row in csv_reader:
        # Verificar se a linha não contém "100" na coluna "Transfer"
        if row[3] != '100 MBytes':  # Ignorar o cabeçalho e filtrar linhas
            csv_writer.writerow(row)

In [11]:
# Escrever a linha modificada no novo arquivo
# Abrir o arquivo CSV modificado para leitura e escrita
with open('ETAPA3_filtered.csv', 'r') as infile, open('ETAPA3_final.csv', 'w', newline='') as outfile:
    csv_reader = csv.reader(infile)
    csv_writer = csv.writer(outfile)

    for row in csv_reader:
        # Remover unidades de Transfer e Bandwidth
        if row[3] != 'Transfer' and row[4] != 'Bandwidth':  # Ignorar o cabeçalho
            row[3] = row[3].replace(' MBytes', '')  # Remover unidade de Transfer
            row[4] = row[4].replace(' Mbits/sec', '')  # Remover unidade de Bandwidth

        # Escrever a linha modificada no novo arquivo
        csv_writer.writerow(row)  

In [12]:
from collections import defaultdict

# Dicionário para armazenar os dados agrupados
data = defaultdict(lambda: {'Transfer': 0, 'Bandwidth': []})

# Abrir o arquivo CSV de entrada e processar os dados
with open('ETAPA3_final.csv', 'r') as infile, open('ETAPA3_aggregated.csv', 'w', newline='') as outfile:
    csv_reader = csv.reader(infile)
    csv_writer = csv.writer(outfile)

    # Ler o cabeçalho
    header = next(csv_reader)
    csv_writer.writerow(['Execution', 'Interval', 'Total Transfer', 'Average Bandwidth'])

    # Processar as linhas do arquivo
    for row in csv_reader:
        execution, interval, transfer, bandwidth = row[0], row[2], float(row[3]), float(row[4])
        key = (execution, interval)
        data[key]['Transfer'] += transfer
        data[key]['Bandwidth'].append(bandwidth)

    # Escrever os dados agregados no novo arquivo
    for (execution, interval), values in data.items():
        total_transfer = values['Transfer']
        avg_bandwidth = sum(values['Bandwidth']) / len(values['Bandwidth'])
        csv_writer.writerow([execution, interval, f"{total_transfer:.2f}", f"{avg_bandwidth:.2f}"])